In [1]:
try:
    import procgen_tools
except ImportError:
    get_ipython().run_line_magic(magic_name='pip', line='install -U git+https://github.com/ulissemini/procgen-tools')

from procgen_tools.utils import setup

setup() # create directory structure and download data 

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/ulissemini/procgen-tools to /tmp/pip-req-build-nbb82qs9
  Running command git clone --filter=blob:none --quiet https://github.com/ulissemini/procgen-tools /tmp/pip-req-build-nbb82qs9
  Resolved https://github.com/ulissemini/procgen-tools to commit dc2243c99110aca92687cdf566daafcfbe7067a0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 9.2 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.2 MB/s eta 0:00:00a 0:00:01
 

HTTPError: 522 Server Error:  for url: https://nerdsniper.net/mats/model_rand_region_5.pth

In [ ]:
from procgen_tools.imports import *
from procgen_tools.procgen_imports import *
from procgen_tools import visualization

# Visualizing details of the goal misgeneralization nets
Let's understand lots of details about [the goal misgeneralization paper](https://arxiv.org/abs/2105.14111). In particular, we'll be looking at the cheese-maze task from the goal misgeneralization task, for which cheese was spawned in the 5x5 top-right corner of the maze. 

Key conclusions:
1. Convolutional layers limit speed of information propagation (_locality_). More precisely, ignoring the effect of `maxpool2D` layers, any portions of the state separated by `n` pixels take at least `ceil(n/2)` convolutional layers to interact.
2. In the maximal maze size of 64 x 64, there is at most **two** steps of computation involving information from e.g. opposite corners of the maze. 

Let's visualize the network structure. Here's a Mermaid diagram. 

![](https://i.imgur.com/5oSHoVQ.png) 

And here's a more dynamic view; small nodes are activations, and large nodes are `nn.Module`s.

In [ ]:

hook.run_with_input(np.zeros((1,3, 64, 64), dtype=np.float32))
hook.get_graph(include_parent_modules=False)

This network is actually substantially overparameterized. The convolutional layers have a `scale` setting, which is set to 4 in Langosco et al.'s trained networks. Training performance and generalization seems similar for `scale=1` and `scale=4`.

In [ ]:
visualization.plot_layer_stats(hook, mode="parameters")
visualization.plot_layer_stats(hook, mode="activations") 

# Viewing activations in the network

In [ ]:
def dummy_obs_pair(color: str, location: Tuple[int, int]=(32,32)):
    """ Returns a mostly-black image pair, the first of which contains a red/green/blue pixel in the center. Returns obs of shape (2, 3, 64, 64). """
    
    assert color in ['R', 'G', 'B'], f'Color must be one of R, G, B, not {color}'
    assert len(location) == 2, 'Location must be a tuple of length 2'
    assert all(0 <= col < 64 for col in location), 'Location must be in [0, 64)'

    channel = {'R': 0, 'G': 1, 'B': 2}[color]
    obs = np.zeros((2, 3, 64, 64), dtype=np.float32)
    obs[0, channel, location[0], location[1]] = 1 # Have one pixel in the middle, in the given channel
    return obs
    
# Let's load a dummy observation with only one nonzero value
plt.imshow(dummy_obs_pair("G")[0].transpose(1,2,0))

In [ ]:
# Get the available labels
hook.run_with_input(dummy_obs_pair('R'))
labels = list(hook.values_by_label.keys())[:-1] # Skip the "_out" layer and remove "embedder." prefixes
assert labels == list(map(expand_label, format_labels(labels)))

In [ ]:
# We can begin to see how information propagates across the net for a dummy single-pixel input
def activ_gen_px(col: int, row: int, layer_name: str, hook, color: str, pair: bool = False):
    """ Get the activations for running a forward pass on a dummy observation, in the given color. Returns shape of (batch, channels, rows, cols), where size batch=1 if pair=False, and batch=2 if pair=True. """
    return get_activations(dummy_obs_pair(color, (row, col)), hook, layer_name)

# Instantiate the plotter
activ_plotter = ActivationsPlotter(labels, plotter=lambda activations, fig: plot_activations(activations[0], fig=fig), activ_gen=activ_gen_px, hook=hook, coords_enabled=True, color="R") # TODO for some reason block2.maxpool_out has a different fig width?
activ_plotter.display()

In [ ]:
# Plot the diff between a single-pixel input and a blank input
activ_plotter = ActivationsPlotter(labels, plotter=lambda activations, fig: plot_activations(activations[0] - activations[1], fig=fig), activ_gen=activ_gen_px, hook=hook, coords_enabled=True, color="R")
activ_plotter.display()

In [ ]:
# Instantiate the plotter
nonzero_plotter = ActivationsPlotter(labels, plotter=plot_nonzero_diffs, activ_gen=activ_gen_px, hook=hook, coords_enabled=True, color="R", pair=True)
nonzero_plotter.display()

# 1: Locality
Consider `n` convolutional layers (3x3 kernel, stride=1, padding=1) which each preserve the height col width of the previous feature maps. The above demonstrates that after these layers, information can only propagate `n` L1 pixels. The network itself is composed of # TODO 

# Visualizing actual observation activations

In [ ]:
default_settings = {'channel_slider': 55, 'label_widget': 'block2.res1.resadd_out'}

In [ ]:
# Show a maze editor side-by-side with the interactive plotter
SEED = 11
venv = create_venv(num=1, start_level=SEED, num_levels=1) # This has to be a single maze, otherwise the vfield wont work
custom_maze_plotter = ActivationsPlotter(labels, lambda activations, fig: plot_activations(activations[0], fig=fig), values_from_venv, hook, defaults=default_settings, venv=venv)

widget_box = custom_vfield(policy, venv=venv, callback=custom_maze_plotter.update_plotter, show_components=True) 
display(widget_box)
    
custom_maze_plotter.display() 
# TODO make it so that it's easy to attach notes to files, load  

# Examine activations with attribution maps


[Integrated gradients](https://arxiv.org/abs/1802.03788) is an attribution method which works by approximating the following integral, where $F : \mathbb{R}^n \rightarrow \mathbb{R}$ is the network, $x$ is the input, $x'$ is the baseline, and $i$ is the feature.

$$
\mathrm{IntegratedGradients}_i(x) = (x_i - x_i') \times \int_{\alpha=0}^1 \frac{\partial F(x' + \alpha (x - x'))}{\partial x_i} d\alpha
$$

[Intuitively](https://www.tensorflow.org/tutorials/interpretability/integrated_gradients) we're accumulating all the change in output from changing $x'$ to $x$ along the linear path $\alpha x' + (1 - \alpha) x$. For a concrete example in words, let $x_i$ be the weight of a heavy exotic frog, $x_i'$ be the average weight over all frogs, $F(x)$ be the estimated jump height of a frog with all the features $x$ (one of which is weight). Given this, the equation above can be interpreted as

$$
\text{Weight gain of exotic frog} \times \text{Average rate jump height changes per unit weight gain} = \text{Jump height change of exotic frog \textbf{due to weight gain}}
$$

In other words, the integrated gradients vector is the vector of the change in output **due** to the change in each input feature.

As an implementation detail, we currently use $x' = \mathbf{0}$ as a baseline, though we'll be changing this soon.

For the interface, <span style="color:blue;">Blue</span> is the activation, brighter for positive, darker for negative. <span style="color:red;">Red</span> is the attribution, brighter for positive, darker for negative.


In [ ]:
from torch.distributions import Categorical
from captum.attr import IntegratedGradients

class WrappedModel(t.nn.Module):
    def __init__(self, hook: cmh.ModuleHook):
        super().__init__()
        assert isinstance(hook, cmh.ModuleHook), "Hook must be a ModuleHook"

        self.hook = hook

    def forward(self, labelled_activation: t.Tensor, obs: t.Tensor, label: str):
        assert isinstance(obs, t.Tensor) and obs.dtype == t.float32, "Obs must be a tensor of dtype float32"
        assert isinstance(labelled_activation, t.Tensor) and labelled_activation.dtype == t.float32, "Labelled activation must be a tensor of dtype float32"

        # could be faster cached but I'm not taking chances with hook reuse bugs
        mask = t.tensor(True) # mask all the things
        patches = {label: cmh.PatchDef(value=labelled_activation, mask=mask)}
        with self.hook.set_hook_should_get_custom_data(), self.hook.use_patches(patches):
            c, _ = self.hook.network(obs)

        # return probs
        return t.stack([c.probs[..., np.array(act_indexes)].sum(-1) for _, act_indexes in models.MAZE_ACTION_INDICES.items()], dim=-1)


model = WrappedModel(hook)
ig = IntegratedGradients(model)

In [ ]:
targets = {k: i for i, k in enumerate(models.MAZE_ACTION_INDICES.keys())}
venv = maze.create_venv(1,0,1)

def viz_attr(target: str, layer_name: str, alpha: float, std_clip_low: float, std_clip_high: float, channel: int = 0, update_counter: int = 0):
    obs = t.tensor(venv.reset(), dtype=t.float32, requires_grad=True)
    # TODO: Baseline/counterfactual for IG, defaults is zero and is not ideal.
    with hook.set_hook_should_get_custom_data():
        hook.network(obs)
        activ = hook.get_value_by_label(layer_name, convert=False)

    attr, delta = ig.attribute(activ, target=targets[target], return_convergence_delta=True, additional_forward_args=(obs, layer_name))
    attr = attr.detach().numpy()

    # Convert to numpy float image format; gaussian normalize to [0,1]
    activ_im = activ[0].detach().numpy()[channel]
    activ_im = (activ_im - activ_im.min()) / (activ_im.max() - activ_im.min())

    attr_im = attr[0][channel]

    # clip outliers
    mean, std = attr_im.mean(), attr_im.std()
    attr_im = np.clip(attr_im, mean - std_clip_low*std, mean + std_clip_high*std)
    # normalize for display
    attr_im = (attr_im - attr_im.min())
    if (attr_im.max() - attr_im.min()) != 0:
        attr_im /= (attr_im.max() - attr_im.min())
    else:
        print("WARNING: attribution failed, image is constant zeros. try a different channel?")

    # TODO: Maybe this should be log scale since model outputs probabilities? maybe model should output log probs?
    # hard to interpret.

    # alternative (non rgb) overlay
    # overlay_im = alpha * attr_im + (1-alpha) * activ_im
    # rb overlay
    green = np.zeros_like(attr_im)
    overlay_im = np.stack([alpha*attr_im, green, (1-alpha)*activ_im], axis=2)

    plt.imshow(overlay_im)


update_counter_widget = widgets.fixed(0) # CURSED

interact(
    viz_attr,
    target=widgets.Dropdown(options=list(targets.keys()), value='RIGHT', description='Target: '),
    layer_name=widgets.Dropdown(options=labels, value=labels[4], description='Label: '),
    alpha=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.5),
    std_clip_low=widgets.FloatSlider(min=0, max=10, step=0.5, value=3), # NOTE: Feel free to make default settings better
    std_clip_high=widgets.FloatSlider(min=0, max=10, step=0.5, value=5),
    channel=widgets.IntSlider(min=0, max=128, step=1, value=0),
    update_counter=update_counter_widget,
);

widget_box = custom_vfield(policy, venv=venv, seed=SEED, callback=lambda _: update_counter_widget.set_trait('value', update_counter_widget.value + 1))
display(widget_box)
